In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats

In [ ]:
def dedup_scar_cell (data_initial):
  #filter scar created by universal repair machinism and scars not caused by dbs
  data1 = data_initial[ ((data_initial['Presence']<=7) & (data_initial['Embryos']<=31)) | (data_initial['p']<0.01)]

  # group the scars that are sequenced multiple times in a cell, drop duplicates
  bool_dup = data1.duplicated(subset=['Cell','Scar'],keep=False)
  not_bool_dup = [not elem for elem in bool_dup]
  my_df = data1[not_bool_dup].sort_values(by='Cell')
  df = my_df[['Library','Cell','Scar']]
  # the actual scar-cell relation 
  df_dedup = df.drop_duplicates()
  
  return df_dedup

In [ ]:
def count_scar_in_cell (df_dedup):
  scar_names = df_dedup.Scar.unique()
  scar_count = len(df_dedup.Scar.unique())

  # Creating a dictionary to store cell count for each scar 
  sc_count_dict = {}
  for i in range(scar_count):
    tempdf = df_dedup[df_dedup['Scar']==scar_names[i]]
    sc_count_dict[scar_names[i]]=(len(tempdf))

  return sc_count_dict

In [ ]:
def scar_pairs (df_dedup):
  scar_pair_df = pd.DataFrame(columns=['Scar1','Scar2']) 
  cell_names = df_dedup.Cell.unique()

  for i in range(N):
    # find all scars in current cell
    temp = df_dedup[df_dedup['Cell']==cell_names[i]].sort_values(by='Scar')
    temp = temp.reset_index()
    if len(df_dedup[df_dedup['Cell']==cell_names[i]])>1:
      for j in range(len(temp.Scar)):
            for k in range(j+1,len(temp.Scar)):
                new_row = {'Scar1':temp.Scar[j], 'Scar2':temp.Scar[k]}
                scar_pair_df = scar_pair_df.append(new_row, ignore_index=True)
  
  scar_pair_df = scar_pair_df.value_counts().rename_axis(['Scar1','Scar2']).reset_index(name='Counts')


  return scar_pair_df

In [ ]:
def remove_sparse_scar (scar_pair_df):
  sc_names_remove0 = pd.concat([scar_pair_df['Scar1'],scar_pair_df['Scar2']]).drop_duplicates()
  return sc_names_remove0

In [ ]:
'''
def remove_false_edge (scar_pair_df,sc_count_dict,N):
  rd = 0.1
  D = rd * N

  count_bad = 0
  count_good = 0
  scar_pair_tested = pd.DataFrame(columns=['Scar1','Scar2']) 
  for i in range(len(scar_pair_df)):
    Nx = sc_count_dict[scar_pair_df.iloc[i]['Scar1']]
    Ny = sc_count_dict[scar_pair_df.iloc[i]['Scar2']]

    dxy = 2 *rd *Nx *Ny /N
    rxy = 2 *rd *Nx *Ny /(N*N)

    p = stats.binom_test(dxy, n=D, p=rxy, alternative='greater')
    if (p<0.01):
      new_row = {'Scar1':scar_pair_df.iloc[i]['Scar1'], 'Scar2':scar_pair_df.iloc[i]['Scar2']}
      scar_pair_tested = scar_pair_tested.append(new_row, ignore_index=True)
      count_good = count_good+1
    if (p>=0.01):
      count_bad = count_bad+1

  return scar_pair_tested
  '''


"\ndef remove_false_edge (scar_pair_df,sc_count_dict,N):\n  rd = 0.1\n  D = rd * N\n\n  count_bad = 0\n  count_good = 0\n  scar_pair_tested = pd.DataFrame(columns=['Scar1','Scar2']) \n  for i in range(len(scar_pair_df)):\n    Nx = sc_count_dict[scar_pair_df.iloc[i]['Scar1']]\n    Ny = sc_count_dict[scar_pair_df.iloc[i]['Scar2']]\n\n    dxy = 2 *rd *Nx *Ny /N\n    rxy = 2 *rd *Nx *Ny /(N*N)\n\n    p = stats.binom_test(dxy, n=D, p=rxy, alternative='greater')\n    if (p<0.01):\n      new_row = {'Scar1':scar_pair_df.iloc[i]['Scar1'], 'Scar2':scar_pair_df.iloc[i]['Scar2']}\n      scar_pair_tested = scar_pair_tested.append(new_row, ignore_index=True)\n      count_good = count_good+1\n    if (p>=0.01):\n      count_bad = count_bad+1\n\n  return scar_pair_tested\n  "

In [ ]:
import statsmodels.stats.multitest as smt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def remove_false_edge (scar_pair_df,sc_count_dict,N):
  rd = 0.1
  D = rd * N

  count_bad = 0
  count_good = 0
  scar_pair_tested = pd.DataFrame(columns=['Scar1','Scar2','p','p_adj']) 
  for i in range(len(scar_pair_df)):
    Nx = sc_count_dict[scar_pair_df.iloc[i]['Scar1']]
    Ny = sc_count_dict[scar_pair_df.iloc[i]['Scar2']]

    dxy = 2 *rd *Nx *Ny /N
    rxy = 2 *rd *Nx *Ny /(N*N)
    Nxy = scar_pair_df.iloc[i]['Counts']

    p = stats.binom_test(Nxy, n=N, p=rxy, alternative='greater')

    new_row = {'Scar1':scar_pair_df.iloc[i]['Scar1'], 'Scar2':scar_pair_df.iloc[i]['Scar2'],'p':p}
    scar_pair_tested = scar_pair_tested.append(new_row, ignore_index=True)

  p_adj = smt.multipletests(scar_pair_tested['p'].to_list(), alpha=0.01, method='fdr_bh')
  scar_pair_tested['p_adj'] = p_adj[1]
  scar_pair_tested = scar_pair_tested[scar_pair_tested['p_adj']<0.01]
  '''
    if (p<=0.01):
      new_row = {'Scar1':scar_pair_df.iloc[i]['Scar1'], 'Scar2':scar_pair_df.iloc[i]['Scar2']}
      scar_pair_tested = scar_pair_tested.append(new_row, ignore_index=True)
      count_good = count_good+1
    if (p>0.01):
      count_bad = count_bad+1
  '''

  return scar_pair_tested

In [ ]:

def remove_false_edge_old (scar_pair_df,sc_count_dict,N):
  rd = 0.1
  D = rd * N

  count_bad = 0
  count_good = 0
  scar_pair_tested = pd.DataFrame(columns=['Scar1','Scar2']) 
  for i in range(len(scar_pair_df)):
    Nx = sc_count_dict[scar_pair_df.iloc[i]['Scar1']]
    Ny = sc_count_dict[scar_pair_df.iloc[i]['Scar2']]

    dxy = 2 *rd *Nx *Ny /N
    rxy = 2 *rd *Nx *Ny /(N*N)
    Nxy = scar_pair_df.iloc[i]['Counts']

    p = stats.binom_test(Nxy, n=N, p=rxy, alternative='greater')
    if (p<0.01):
      new_row = {'Scar1':scar_pair_df.iloc[i]['Scar1'], 'Scar2':scar_pair_df.iloc[i]['Scar2']}
      scar_pair_tested = scar_pair_tested.append(new_row, ignore_index=True)
      count_good = count_good+1
    if (p>=0.01):
      count_bad = count_bad+1
  

  return scar_pair_tested


In [ ]:
def scar_degree (scar_pair_df,scar_names):
  scar_connectivity = pd.DataFrame(columns=['Scar','Connectivity']) 
  for i in scar_names:
    new_row = {'Scar': i, 'Connectivity': len(scar_pair_df[scar_pair_df['Scar1']==i])+len(scar_pair_df[scar_pair_df['Scar2']==i])} 
    scar_connectivity = scar_connectivity.append(new_row, ignore_index=True)

  scar_connectivity.sort_values(by='Connectivity', ascending=False, inplace=True)
  return scar_connectivity

In [ ]:
def force_1c (scar_pair_df,scar_connectivity,scar_names):
  top = scar_connectivity.iloc[0]['Scar']
  scar_pair_df_1c = scar_pair_df
  for i in scar_names:
    if i != top:
      if i<top:
        #i=scar1,top=scar2
        new_row = {'Scar1':i, 'Scar2':top}
        scar_pair_df_1c = scar_pair_df_1c.append(new_row, ignore_index=True)
      else: 
        new_row = {'Scar1':top, 'Scar2':i}
        scar_pair_df_1c = scar_pair_df_1c.append(new_row, ignore_index=True)
  
  scar_pair_df_1c=scar_pair_df_1c.drop_duplicates()
  return scar_pair_df_1c

In [ ]:
def force_1c_0_as_Root (scar_pair_df,scar_connectivity,scar_names):
  top = '0'
  scar_pair_df_1c = scar_pair_df
  for i in scar_names:
      new_row = {'Scar1':top, 'Scar2':i}
      scar_pair_df_1c = scar_pair_df_1c.append(new_row, ignore_index=True)
  
  scar_pair_df_1c=scar_pair_df_1c.drop_duplicates()
  return scar_pair_df_1c

In [ ]:
def final_format (scar_pair_df_1c,sc_count_dict):
  final_scar_pair_df = pd.DataFrame(columns=['Scar1','Scar2','cellCount1','cellCount2'])
  for i in range(len(scar_pair_df_1c)):
    scar1 = scar_pair_df_1c.iloc[i]['Scar1']
    scar2 = scar_pair_df_1c.iloc[i]['Scar2']
    if scar1=='0':
      count1 = 10000
    else:
      count1 = sc_count_dict[scar1]
    count2 = sc_count_dict[scar2]
    new_row = {'Scar1': scar1, 'Scar2':scar2,'cellCount1': count1,'cellCount2':count2}
    final_scar_pair_df = final_scar_pair_df.append(new_row, ignore_index=True)
  return final_scar_pair_df

## Test

##**RUN**



In [ ]:
d5 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A5_scars_compared.csv")
d6 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A6_scars_compared.csv")
d7 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A7_scars_compared.csv")
data_initial = pd.concat([d5,d6,d7])
#data_initial = d5
#data_initial = d6
#data_initial = d7

In [ ]:
data_initial = data_initial[(data_initial['Library']=='P7exo')|(data_initial['Library']=='P7endo')|(data_initial['Library']=='P5')|(data_initial['Library']=='P6')]
#data_initial = data_initial[(data_initial['Library']=='B7')|(data_initial['Library']=='B5')|(data_initial['Library']=='B6')]
#data_initial = data_initial[(data_initial['Library']=='H7')|(data_initial['Library']=='H5')|(data_initial['Library']=='H6')]

#data_initial = data_initial[(data_initial['Library']=='H5')]
#data_initial = data_initial[(data_initial['Library']=='B5')] ### No valid scar after testing 
#data_initial = data_initial[(data_initial['Library']=='P5')]

#data_initial = data_initial[(data_initial['Library']=='H6')]
#data_initial = data_initial[(data_initial['Library']=='B6')]. ### NO VALID SCAR even before testing
#data_initial = data_initial[(data_initial['Library']=='P6')]

#data_initial = data_initial[(data_initial['Library']=='H7')]
#data_initial = data_initial[(data_initial['Library']=='B7')]
#data_initial = data_initial[(data_initial['Library']=='P7exo')|(data_initial['Library']=='P7endo')]
#data_initial = data_initial[(data_initial['Library']=='P7endo')]

In [ ]:
# Find out unique cell-scar combination
df_dedup = dedup_scar_cell(data_initial)

# total cell count (before filtering)
N = len(df_dedup.Cell.unique())

# cell count for each scar
sc_count_dict = count_scar_in_cell (df_dedup)

# scar1-scar2 pairs 
scar_pair_df = scar_pairs (df_dedup)

# remove edges resulting from doublets 
scar_pair_tested = remove_false_edge (scar_pair_df,sc_count_dict,N)



# remove sparse scar
scar_names_remove0 = remove_sparse_scar (scar_pair_df)
scar_names_tested_remove0 = remove_sparse_scar (scar_pair_tested)


# calculate scar connectivity (to find the root scar)
scar_connectivity = scar_degree (scar_pair_df,scar_names_remove0)
scar_tested_connectivity = scar_degree (scar_pair_tested,scar_names_tested_remove0)


# force connect to root node
'''old
scar_pair_df_1c = force_1c (scar_pair_df,scar_connectivity,scar_names_remove0)
scar_pair_df_tested_1c = force_1c (scar_pair_tested,scar_tested_connectivity,scar_names_tested_remove0)
'''

scar_pair_df_1c = force_1c_0_as_Root (scar_pair_df,scar_connectivity,scar_names_remove0)
scar_pair_df_tested_1c = force_1c_0_as_Root (scar_pair_tested,scar_tested_connectivity,scar_names_tested_remove0)


# add cell count to df 
final_scar_pair_df = final_format (scar_pair_df_1c,sc_count_dict)
final_scar_pair_tested_df = final_format (scar_pair_df_tested_1c,sc_count_dict)


#scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/scar_pair_A567_Pancreas.csv")
#final_scar_pair_tested_df


In [ ]:
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version2/A567_H_all.csv")
final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version2/A567_P_tested.csv")

## LiuShuiXian

In [ ]:
d5 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A5_scars_compared.csv")
d6 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A6_scars_compared.csv")
d7 = pd.read_csv("/content/drive/My Drive/Year Four/ZB4171 Project/GSE106121_A7_scars_compared.csv")
#No need# data_initial = pd.concat([d5,d6,d7])
#data_initial = d5
data_initial = d6
#data_initial = d7

#data_initial = data_initial[(data_initial['Library']=='P7exo')|(data_initial['Library']=='P7endo')|(data_initial['Library']=='P5')|(data_initial['Library']=='P6')]
#data_initial = data_initial[(data_initial['Library']=='B7')|(data_initial['Library']=='B5')|(data_initial['Library']=='B6')]
#data_initial = data_initial[(data_initial['Library']=='H7')|(data_initial['Library']=='H5')|(data_initial['Library']=='H6')]

#data_initial = data_initial[(data_initial['Library']=='H5')]
#data_initial = data_initial[(data_initial['Library']=='B5')] ### No valid scar after testing 
#data_initial = data_initial[(data_initial['Library']=='P5')]

data_initial = data_initial[(data_initial['Library']=='H6')]
#data_initial = data_initial[(data_initial['Library']=='B6')]. ### NO VALID SCAR even before testing
#data_initial = data_initial[(data_initial['Library']=='P6')]

#data_initial = data_initial[(data_initial['Library']=='H7')]
#data_initial = data_initial[(data_initial['Library']=='B7')]
#data_initial = data_initial[(data_initial['Library']=='P7exo')|(data_initial['Library']=='P7endo')]
#data_initial = data_initial[(data_initial['Library']=='P7endo')]



# Find out unique cell-scar combination
df_dedup = dedup_scar_cell(data_initial)

# total cell count (before filtering)
N = len(df_dedup.Cell.unique())

# cell count for each scar
sc_count_dict = count_scar_in_cell (df_dedup)

# scar1-scar2 pairs 
scar_pair_df = scar_pairs (df_dedup)

# remove edges resulting from doublets 
scar_pair_tested = remove_false_edge (scar_pair_df,sc_count_dict,N)
scar_pair_tested_old = remove_false_edge_old (scar_pair_df,sc_count_dict,N)


# remove sparse scar
scar_names_remove0 = remove_sparse_scar (scar_pair_df)
scar_names_tested_remove0 = remove_sparse_scar (scar_pair_tested)


# calculate scar connectivity (to find the root scar)
scar_connectivity = scar_degree (scar_pair_df,scar_names_remove0)
scar_tested_connectivity = scar_degree (scar_pair_tested,scar_names_tested_remove0)


# force connect to root node
'''old
scar_pair_df_1c = force_1c (scar_pair_df,scar_connectivity,scar_names_remove0)
scar_pair_df_tested_1c = force_1c (scar_pair_tested,scar_tested_connectivity,scar_names_tested_remove0)
'''

scar_pair_df_1c = force_1c_0_as_Root (scar_pair_df,scar_connectivity,scar_names_remove0)
scar_pair_df_tested_1c = force_1c_0_as_Root (scar_pair_tested,scar_tested_connectivity,scar_names_tested_remove0)


# add cell count to df 
final_scar_pair_df = final_format (scar_pair_df_1c,sc_count_dict)
final_scar_pair_tested_df = final_format (scar_pair_df_tested_1c,sc_count_dict)

#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version2/A7_H_all.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version2/A5_HPBtested.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version2/A5_H_tested_NEW.csv")

new = final_scar_pair_tested_df[(final_scar_pair_tested_df['cellCount1']>=20) & (final_scar_pair_tested_df['cellCount2']>=20)]
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version3/A5_P.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version3/test_A6_all.csv")

In [ ]:
final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/Version3/test.csv")

In [ ]:
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_H_total.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_H_tested.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A7_all_tested.csv")

#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_B_total.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_B_tested.csv")

#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_P_total.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A567_P_tested.csv")

#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A7_H_tested.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A7_P_tested.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A7_B.csv")

#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A6_H_tested.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A6_P.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A6_B.csv") ## NO SCAR

#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A5_H_tested.csv")
#final_scar_pair_tested_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A5_P_tested.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A5_P.csv")
#final_scar_pair_df.to_csv("/content/drive/My Drive/Year Four/ZB4171 Project/ScarPair_A5_B.csv")

### Exploration of Poisson binomial (Not used, exceed RAM)

In [ ]:
scar_names = scar_names_tested_remove0
scar_connectivity = scar_tested_connectivity
t_count = 0
for i in scar_names:
  t_count = t_count + sc_count_dict[i]

In [ ]:
for i in scar_names:
  pij_list = []
  for j in scar_names:
    if i!=j:
      pij = 1 - (1-(sc_count_dict[i]/t_count))**sc_count_dict[j]
      pij_list = pij_list + [pij]
  print(pij_list)

[0.9999439993807856, 0.9999999986403519, 0.9999999927665613, 0.9999926426636504, 0.9991274850254814, 0.9999652633765148, 0.999989473729366, 0.9999097186474174]
[0.9999361690636115, 0.9999998527030705, 0.9999994660226139, 0.9998891479529387, 0.995606333751805, 0.9996334604814452, 0.9998539169026597, 0.999234866046896]
[0.9999999994142674, 0.9999999383386736, 0.9999999999999843, 0.9999999980263986, 0.999993508780007, 0.9999999725649237, 0.9999999963772409, 0.9999998614139386]
[0.9999999960380578, 0.9999997256112154, 0.9999999999999792, 0.9999999880318321, 0.9999809968947736, 0.9999998686933668, 0.9999999791988454, 0.9999994266153717]
[0.9999923288102809, 0.9998987671729823, 0.9999999953265933, 0.9999999775274745, 0.9986640813692199, 0.9999353663037905, 0.9999789475297863, 0.9998414436142908]
[0.9989501485963278, 0.9952832518388517, 0.9999859170414892, 0.9999648539230626, 0.998446364629659, 0.9963678661519866, 0.9981100090124492, 0.9938747540298207]
[0.9999610788654599, 0.9996401306496268

In [ ]:
scar_dr = pd.DataFrame(columns=['Scar','N','Prob']) 
#pij_list = []
for i in scar_names:
  pij_list = []
  for j in scar_names:
    if i!=j:
      pij = 1 - (1-(sc_count_dict[i]/t_count))**sc_count_dict[j]
      pij_list = pij_list + [pij]
  # perform poisson binomial test
  pb = PoiBin(pij_list)
  obs_deg = scar_connectivity[scar_connectivity['Scar']==i].iloc[0,1]
  new_row = {'Scar': i, 'N': sc_count_dict[i], 'Prob':pb.cdf(obs_deg)} 
  scar_dr= scar_dr.append(new_row, ignore_index=True)

scar_dr.sort_values(by='Prob', ascending=False)

,Scar,N,Prob
2,13:46M6D29M,171,1.000000e+00
3,32:75M,157,1.961006e-11
0,1033:51M1I23M,105,8.975447e-16
4,28:44M9D31M,99,8.163485e-16
7,361:50M7I18M,96,8.119333e-16
8,779:34S41M,78,4.050579e-16
6,44:44M3D31M,86,3.883612e-16
1,1202:49M26S,82,1.569925e-16
5,624:43M1I4M3D27M,59,7.979728e-17


,Sequence,Barcode,Library,Cell,Reads,CIGAR,UMI,Presence,p,Embryos,Scar


In [ ]:
scar_connectivity

In [ ]:
sc_count_dict['44:44M3D31M']

In [ ]:
sc_count_dict['361:50M7I18M']

In [ ]:
sc_count_dict['1033:51M1I23M']

In [ ]:
### Pbinom 
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 29, 2016
Module:
    poibin - Poisson Binomial Distribution
Author:
    Mika Straka
Description:
    Implementation of the Poisson Binomial distribution for the sum of
    independent and not identically distributed random variables as described
    in the reference [Hong2013]_.
    Implemented method:
        * ``pmf``: probability mass function
        * ``cdf``: cumulative distribution function
        * ``pval``: p-value (1 - cdf)
Usage:
    Be ``p`` a list or  NumPy array of success probabilities for ``n``
    non-identically distributed Bernoulli random variables.
    Import the module and create an instance of the distribution with::
        >>> from poibin import PoiBin
        >>> pb = PoiBin(p)
    Be ``x`` a list or NumPy array of different number of successes.
    To obtain the:
    * probability mass function of x, use::
        >>> pb.pmf(x)
    * cumulative distribution function of x, use::
        >>> pb.cdf(x)
    * p-values of x, use::
        >>> pb.pval(x)
    The functions are applied component-wise and a NumPy array of the same
    length as ``x`` is returned.
References:
.. [Hong2013] Yili Hong, On computing the distribution function for the Poisson
    binomial distribution,
    Computational Statistics & Data Analysis, Volume 59, March 2013,
    Pages 41-51, ISSN 0167-9473,
    http://dx.doi.org/10.1016/j.csda.2012.10.006.
"""

import collections
import numpy as np


class PoiBin(object):
    """Poisson Binomial distribution for random variables.
    This class implements the Poisson Binomial distribution for Bernoulli
    trials with different success probabilities. The distribution describes
    thus a random variable that is the sum of independent and not identically
    distributed single Bernoulli random variables.
    The class offers methods for calculating the probability mass function, the
    cumulative distribution function, and p-values for right-sided testing.
    """

    def __init__(self, probabilities):
        """Initialize the class and calculate the ``pmf`` and ``cdf``.
        :param probabilities: sequence of success probabilities :math:`p_i \\in
            [0, 1] \\forall i \\in [0, N]` for :math:`N` independent but not
            identically distributed Bernoulli random variables
        :type probabilities: numpy.array
        """
        self.success_probabilities = np.array(probabilities)
        self.number_trials = self.success_probabilities.size
        self.check_input_prob()
        self.omega = 2 * np.pi / (self.number_trials + 1)
        self.pmf_list = self.get_pmf_xi()
        self.cdf_list = self.get_cdf(self.pmf_list)

# ------------------------------------------------------------------------------
# Methods for the Poisson Binomial Distribution
# ------------------------------------------------------------------------------

    def pmf(self, number_successes):
        """Calculate the probability mass function ``pmf`` for the input values.
        The ``pmf`` is defined as
        .. math::
            pmf(k) = Pr(X = k), k = 0, 1, ..., n.
        :param number_successes: number of successful trials for which the
            probability mass function is calculated
        :type number_successes: int or list of integers
        """
        self.check_rv_input(number_successes)
        return self.pmf_list[number_successes]

    def cdf(self, number_successes):
        """Calculate the cumulative distribution function for the input values.
        The cumulative distribution function ``cdf`` for a number ``k`` of
        successes is defined as
        .. math::
            cdf(k) = Pr(X \\leq k), k = 0, 1, ..., n.
        :param number_successes: number of successful trials for which the
            cumulative distribution function is calculated
        :type number_successes: int or list of integers
        """
        self.check_rv_input(number_successes)
        return self.cdf_list[number_successes]

    def pval(self, number_successes):
        """Return the p-values corresponding to the input numbers of successes.
        The p-values for right-sided testing are defined as
        .. math::
            pval(k) = Pr(X \\geq k ),  k = 0, 1, ..., n.
        .. note::
            Since :math:`cdf(k) = Pr(X <= k)`, the function returns
            .. math::
                1 - cdf(X < k) & = 1 - cdf(X <= k - 1)
                               & = 1 - cdf(X <= k) + pmf(X = k),
                               k = 0, 1, .., n.
        :param number_successes: number of successful trials for which the
            p-value is calculated
        :type number_successes: int, numpy.array, or list of integers
        """
        self.check_rv_input(number_successes)
        i = 0
        try:
            isinstance(number_successes, collections.Iterable)
            pvalues = np.array(number_successes, dtype='float')
            # if input is iterable (list, numpy.array):
            for k in number_successes:
                pvalues[i] = 1. - self.cdf(k) + self.pmf(k)
                i += 1
            return pvalues
        except TypeError:
            # if input is an integer:
            if number_successes == 0:
                return 1
            else:
                return 1 - self.cdf(number_successes - 1)

# ------------------------------------------------------------------------------
# Methods to obtain pmf and cdf
# ------------------------------------------------------------------------------

    def get_cdf(self, event_probabilities):
        """Return the values of the cumulative density function.
        Return a list which contains all the values of the cumulative
        density function for :math:`i = 0, 1, ..., n`.
        :param event_probabilities: array of single event probabilities
        :type event_probabilities: numpy.array
        """
        cdf = np.empty(self.number_trials + 1)
        cdf[0] = event_probabilities[0]
        for i in range(1, self.number_trials + 1):
            cdf[i] = cdf[i - 1] + event_probabilities[i]
        return cdf

    def get_pmf_xi(self):
        """Return the values of the variable ``xi``.
        The components ``xi`` make up the probability mass function, i.e.
        :math:`\\xi(k) = pmf(k) = Pr(X = k)`.
        """
        chi = np.empty(self.number_trials + 1, dtype=complex)
        chi[0] = 1
        half_number_trials = int(
            self.number_trials / 2 + self.number_trials % 2)
        # set first half of chis:
        chi[1:half_number_trials + 1] = self.get_chi(
            np.arange(1, half_number_trials + 1))
        # set second half of chis:
        chi[half_number_trials + 1:self.number_trials + 1] = np.conjugate(
            chi[1:self.number_trials - half_number_trials + 1] [::-1])
        chi /= self.number_trials + 1
        xi = np.fft.fft(chi)
        if self.check_xi_are_real(xi):
            xi = xi.real
        else:
            raise TypeError("pmf / xi values have to be real.")
        xi += np.finfo(type(xi[0])).eps
        return xi

    def get_chi(self, idx_array):
        """Return the values of ``chi`` for the specified indices.
        :param idx_array: array of indices for which the ``chi`` values should
            be calculated
        :type idx_array: numpy.array
        """
        # get_z:
        exp_value = np.exp(self.omega * idx_array * 1j)
        xy = 1 - self.success_probabilities + \
            self.success_probabilities * exp_value[:, np.newaxis]
        # sum over the principal values of the arguments of z:
        argz_sum = np.arctan2(xy.imag, xy.real).sum(axis=1)
        # get d value:
        exparg = np.log(np.abs(xy)).sum(axis=1)
        d_value = np.exp(exparg)
        # get chi values:
        chi = d_value * np.exp(argz_sum * 1j)
        return chi

# ------------------------------------------------------------------------------
# Auxiliary functions
# ------------------------------------------------------------------------------

    def check_rv_input(self, number_successes):
        """Assert that the input values ``number_successes`` are OK.
        The input values ``number_successes`` for the random variable have to be
        integers, greater or equal to 0, and smaller or equal to the total
        number of trials ``self.number_trials``.
        :param number_successes: number of successful trials
        :type number_successes: int or list of integers """
        try:
            for k in number_successes:
                assert (type(k) == int or type(k) == np.int64), \
                        "Values in input list must be integers"
                assert k >= 0, 'Values in input list cannot be negative.'
                assert k <= self.number_trials, \
                    'Values in input list must be smaller or equal to the ' \
                    'number of input probabilities "n"'
        except TypeError:
            assert (type(number_successes) == int or \
                type(number_successes) == np.int64), \
                'Input value must be an integer.'
            assert number_successes >= 0, "Input value cannot be negative."
            assert number_successes <= self.number_trials, \
                'Input value cannot be greater than ' + str(self.number_trials)
        return True

    @staticmethod
    def check_xi_are_real(xi_values):
        """Check whether all the ``xi``s have imaginary part equal to 0.
        The probabilities :math:`\\xi(k) = pmf(k) = Pr(X = k)` have to be
        positive and must have imaginary part equal to zero.
        :param xi_values: single event probabilities
        :type xi_values: complex
        """
        return np.all(xi_values.imag <= np.finfo(float).eps)

    def check_input_prob(self):
        """Check that all the input probabilities are in the interval [0, 1]."""
        if self.success_probabilities.shape != (self.number_trials,):
            raise ValueError(
                "Input must be an one-dimensional array or a list.")
        if not np.all(self.success_probabilities >= 0):
            raise ValueError("Input probabilities have to be non negative.")
        if not np.all(self.success_probabilities <= 1):
            raise ValueError("Input probabilities have to be smaller than 1.")

################################################################################
# Main
################################################################################

if __name__ == "__main__":
    pass
